# NDVI for a single pixel overtime

This Notebook calcualtes NDVI for a single pixel based on WGS84 coordinates as supplied by the user. 

## Limitations
Work must still be done to remove cloud affected pixels. Currently the scrip only fetches pixels from scenes with less than a specified cloud percentage. However, this should be improved to integrated the cloud mask for said pixel. 

### Purpose
This is not a generic tool but was develop for a specfic project. Work is still to be done to increase its genericness 

In [ ]:
import pystac_client
import stackstac
import matplotlib.pyplot as plt
from dask.distributed import Client, progress
import planetary_computer
import numpy as np
import rioxarray
from datetime import datetime
import xarray as xr


In [ ]:
#AS per > https://github.com/microsoft/PlanetaryComputerExamples/blob/main/datasets/sentinel-2-l2a/baseline-change.ipynb

def harmonize_to_old(data):
    """
    Harmonize new Sentinel-2 data to the old baseline.

    Parameters
    ----------
    data: xarray.DataArray
        A DataArray with four dimensions: time, band, y, x

    Returns
    -------
    harmonized: xarray.DataArray
        A DataArray with all values harmonized to the old
        processing baseline.
    """
    cutoff = datetime(2022, 1, 25)
    offset = 1000
    bands = [
        "red",
        "nir",
    ]

    old = data.sel(time=slice(cutoff))

    to_process = list(set(bands) & set(data.band.data.tolist()))
    new = data.sel(time=slice(cutoff, None)).drop_sel(band=to_process)

    new_harmonized = data.sel(time=slice(cutoff, None), band=to_process).clip(offset)
    new_harmonized -= offset

    new = xr.concat([new, new_harmonized], "band").sel(band=data.band.data.tolist())
    return xr.concat([old, new], dim="time")

In [ ]:
client = Client()
print(f"/proxy/{client.scheduler_info()['services']['dashboard']}/status")

In [ ]:
catalog = pystac_client.Client.open(
    #"http#s://earth-search.aws.element84.com/v1"
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

<div style="background-color: #03fc62; padding: 10px;">
    
###  User Input Required
* **time_of_interest**: The date range of imagery to fetch (in format of yyyy-mm-dd/yyyy-mm-dd)
* **point_lon, point_lat**: longitude and latitude of pixel to interrogate

</div>


In [ ]:
time_of_interest = "2017-02-01/2023-10-24"
point_lon, point_lat = 175.9538129505,-37.6752894183

In [ ]:
geom=[point_lon-0.000005, point_lat-0.000005, point_lon+0.000005, point_lat+0.000005]
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=dict(type="Point", coordinates=(point_lon, point_lat)),
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 50}},
)

matched = search.matched()
if matched is not None:
    print(f"{search.matched()} scenes found")


In [ ]:
# get all items
items_dict = search.item_collection()
items_dict

In [ ]:
stack = (
    stackstac.stack(
        items_dict,
        assets=["B04", "B08"],  # blue, green, red, nir
        chunksize=256,  # set chunk size to 256 to get one chunk per time step
        bounds_latlon=geom,
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
    .assign_coords(
        band=lambda x: x.common_name.rename("band"),  # use common names
        #time=lambda x: x.time.dt.round(
        #    "D"
        #),  # round time to daily for nicer plot labels
    )
)

In [ ]:
# Correct for S2 offset
stack = harmonize_to_old(stack)

In [ ]:
subset = stack.compute()

In [ ]:
nir, red = subset.sel(band="nir"), subset.sel(band="red")
subset['ndvi'] = (nir - red) / (nir + red)


In [ ]:
subset = subset['ndvi']
subset = subset.drop([c for c in subset.coords if not (c in ['time', 'x', 'x'])])


# NDVI Scatter Plot

In [ ]:
fig, ax = plt.subplots(figsize=(30, 12)) 

fig.suptitle('NDVI For Selected Pixel Over Time', fontsize=40)

# Add ant grey windows here
date_ranges = [
    ('15/02/2023', '28/04/2023'),
    ('15/02/2022', '28/04/2022'),
    ('15/02/2021', '28/04/2021'),
    ('15/02/2020', '28/04/2020'),
    ('15/02/2019', '28/04/2019'),
    ('15/02/2018', '28/04/2018'),
    ('15/02/2017', '28/04/2017')

]

for begin_date_str, end_date_str in date_ranges:
    detect_begin = datetime.strptime(begin_date_str, '%d/%m/%Y')
    detect_end = datetime.strptime(end_date_str, '%d/%m/%Y')  
    ax.axvspan(detect_begin, detect_end, facecolor='grey', alpha=0.2)

cyclone = datetime(2023, 2, 5)

plt.axvline(x=cyclone, color='red', linestyle='--', label='Cyclone')


subset.plot(marker='o', linestyle='',  alpha=0.8, markersize=10,  ax=ax)


# Linear Tredline

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import linregress
fig, ax = plt.subplots(figsize=(30, 12)) 

fig.suptitle('NDVI For Selected Pixel Over Time', fontsize=40)

# Add ant grey windows here
date_ranges = [
    ('15/02/2023', '31/03/2023'),
    ('15/02/2022', '31/03/2022'),
    ('15/02/2021', '31/03/2021'),
    ('15/02/2020', '31/03/2020'),
    ('15/02/2019', '31/03/2019'),
    ('15/02/2018', '31/03/2018'),
    ('15/02/2017', '31/03/2017')

]

for begin_date_str, end_date_str in date_ranges:
    detect_begin = datetime.strptime(begin_date_str, '%d/%m/%Y')
    detect_end = datetime.strptime(end_date_str, '%d/%m/%Y')
    
    ax.axvspan(detect_begin, detect_end, facecolor='grey', alpha=0.2)
#subset.plot()

ndvi_array = subset.values.ravel()

# Create an array of time values
time_values = np.arange(len(ndvi_array))

# Perform linear regression
slope, intercept, r_value, p_value, std_err = linregress(time_values, ndvi_array)

# Create a trendline based on the linear regression results
trendline = slope * time_values + intercept


subset.plot(marker='o', linestyle='',  alpha=0.8, markersize=10,  ax=ax)
plt.plot(subset.time, trendline, color='red', label='Trendline')


# None Linear Trend line

In [ ]:
# def quadratic(x, a, b, c):
#     return a * x**2 + b * x + c

# from scipy.optimize import curve_fit

# ndvi_array = subset.values.ravel()

# # Create an array of time values
# time_values = np.arange(len(ndvi_array))

# # Fit the data to the quadratic function
# params, covariance = curve_fit(quadratic, time_values, ndvi_array)
# x_fit = np.linspace(min(time_values), max(time_values), 100)
# y_fit = quadratic(x_fit, *params)
